# Travelling salesman problem

In [0]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
import seaborn as sns
sns.set()
import numpy as np
from random import shuffle
import random

def generate_problem(N_points, dimension = 2):
    return np.random.rand(N_points, dimension)

def plot_problem(X):
    plt.figure(figsize=(10,6))
    N_points, dimension = X.shape
    if dimension > 2:
        print('I can plot only 2d pictures now. 🐱')

    x, y = X.T
    plt.scatter(x,y, marker = '^', s=160, label = 'Houses')
    plt.title('The village')
    plt.legend()
    plt.savefig('salesman_problem.svg')
    plt.show()

def plot_solution(X, creature, generation = ''):
    N_points, dimension = X.shape
    if dimension > 2:
        print('I can plot only 2d pictures now. 🐱')

    x, y = X.T
    plt.figure(figsize=(10,6))
    plt.scatter(x,y, marker = '^', s=160, label = 'Houses')
    plt.title('The village')
    X_new = np.array(X)[creature]
    x, y = X_new.T
    plt.plot(x,y, 'r-', label='Salesman path ({:.2f})'.format(distance(X, creature)))
    plt.legend()
    plt.savefig('gen{}.png'.format(generation))
    plt.show()

def distance(X, y):
    N_points, dimension = X.shape
    dist = 0
    for i in range(N_points-1):
        dist += np.linalg.norm(X[y[i]] - X[y[i+1]])
    return(dist)

def evaluate_generation(X, population, full=False):
    distances = [distance(X, creature) for creature in population]
    if not full:
        idmin = np.argmin(np.array(distances))
        return idmin, distances[idmin]
    else:
        best = np.array(distances).argsort()
        return np.array(population)[best]


def random_solution(X):
    N_points, dimension = X.shape
    y = list(np.arange(N_points))
    shuffle(y)
    # print('Distance {:.2f}'.format(distance(X, y)))
    return y

def create_population(X, N_creatures = 500):
    creatures_list = [random_solution(X) for i in range(N_creatures)]
    return creatures_list

def mutation(population):
    mutation_rate = 0.1
    shuffle(population)
    N_creatures = len(population)
    N_points = len(population[0])
    mutated = random.sample(range(N_creatures), int(mutation_rate*N_creatures))

    for i in mutated:
        creature = population[i]
        for i in range(int(mutation_rate*N_points)):
            random_index1, random_index2 = random.sample(range(N_points), 2)
            temp = creature[random_index1]
            creature[random_index1] = creature[random_index2]
            creature[random_index2] = temp
        population[i] = creature
    return population

def children_creation(parent1, parent2):
    # http://www.rubicite.com/Tutorials/GeneticAlgorithms/CrossoverOperators/Order1CrossoverOperator.aspx
    child = []
    childP1 = []
    childP2 = []
    
    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))
    
    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):
        childP1.append(parent1[i])
        
    childP2 = [item for item in parent2 if item not in childP1]

    child = childP1 + childP2
    return child

def breed(population):
    shuffle(population)
    N_creatures = len(population)
    for i in range(N_creatures//2):
        children = children_creation(population[i], population[i+1])
        population.append(children)
    return population

def selection(X, population, offsprings = 500):
    distances = [distance(X, creature) for creature in population]
    best = np.array(distances).argsort()[:offsprings].tolist()
    return np.array(population)[best].tolist()

def let_eat_bee(N_points, N_creatures, N_generations):
    Loss = []
    generations = []
    X = generate_problem(N_points)
    plot_problem(X)
    population = create_population(X, N_creatures)
    for generation in range(N_generations):
        population = mutation(population)
        population = breed(population)
        population = selection(X, population, offsprings = N_creatures)
 
        if generation % 10 == 0:
            idp, min_dist = evaluate_generation(X, population)
            Loss.append(min_dist)
            generations.append(generation)
            plot_solution(X, population[idp], generation)
    
    plt.show()
    plt.figure(figsize=(10,6))
    plt.semilogy(generations, Loss)
    plt.xlabel('generation')
    plt.ylabel('Path length')
    plt.savefig('loss.svg')
    

In [0]:
let_eat_bee(40, 100, 2000)

Output hidden; open in https://colab.research.google.com to view.